In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from models import DoublePointerNetwork, Critic
from utils import generate_batch_qap_problem_instance

In [2]:
class Config():
    """The config class."""
    num_instances = 20 # (n)
    loc_input_dim = 2 
    fac_input_dim = 20 # one-hot encoding of the facilities as initial input feature
    attn_dim = 256
    embed_dim = 512
    dropout = 0.1
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    batch_size = 64
    epochs = 20
    optimizer = 'adam' # 'sgd'

In [3]:
config = Config()

In [4]:
model = DoublePointerNetwork(
    num_instances=config.num_instances, 
    loc_input_dim=config.loc_input_dim, 
    fac_input_dim=config.fac_input_dim, 
    embed_dim=config.embed_dim, 
    attn_dim=config.attn_dim, 
    dropout=config.dropout, 
    device=config.device
    )

critic = Critic(
            2*(config.num_instances**2)+2*config.num_instances, # 2*(n^2) + 2*n
            config.embed_dim, 
            config.dropout
            )

In [5]:
locations, _, distance_matrix, fac_matrix =\
                generate_batch_qap_problem_instance(config.batch_size, config.num_instances)



In [6]:
print(locations.shape, distance_matrix.shape, fac_matrix.shape)

torch.Size([64, 20, 2]) torch.Size([64, 20, 20]) torch.Size([64, 20, 20])


In [7]:
locations.float().dtype

torch.float32

In [8]:
batch_size = fac_matrix.size(0)
num_instances = fac_matrix.size(1)

# perform batch rollout
U_probs, L_probs, U_selected_locs, L_selected_facs =\
    model(fac_matrix=fac_matrix.float(),
          locations=locations.float()
          )


In [9]:
def to_sparse_batch(x, adj):
    """Converts a dense adjacency matrix to a sparse batch of adjacency matrices.
    """
    batch_size = x.size(0)
    num_nodes = x.size(1)
    

    x = x.reshape(-1, x.shape[-1])
    # get edge indexs
    indices = adj.nonzero()
    edge_indexs = indices[:,1:].t()
    batch_offset = indices[:,0]*num_nodes
    offset = torch.stack((batch_offset, batch_offset))
    edge_indexs = edge_indexs + offset
    # get batch indicator
    batch_ls = [ [j for i in range(num_nodes)] for j in range(batch_size)]
    batch = torch.tensor(batch_ls, dtype=torch.int64)
    batch = batch.reshape(-1)
    return x, edge_indexs, batch

In [34]:
indices = fac_matrix.nonzero()

In [35]:
edge_indexs = indices[:,1:].t()

In [36]:
edge_indexs

tensor([[ 0,  0,  0,  ..., 19, 19, 19],
        [ 0,  1,  2,  ..., 17, 18, 19]])

In [37]:
num_node = num_instances
batch_offset = indices[:,0]*num_node

In [38]:
batch_offset

tensor([   0,    0,    0,  ..., 1260, 1260, 1260])

In [39]:
offset = torch.stack((batch_offset, batch_offset))
edge_indexs = edge_indexs + offset

In [41]:
batch_ls = [ [j for i in range(num_node)] for j in range(batch_size)]

In [43]:
batch = torch.tensor(batch_ls, dtype=torch.int64).cuda()
batch = batch.reshape(-1)

In [44]:
batch

tensor([ 0,  0,  0,  ..., 63, 63, 63], device='cuda:0')